In [6]:
import cv2
import numpy as np

# -------------------------- 一、公路图片直线检测（修复路径+读取判断）--------------------------
def detect_road_lines(image_path):
    # 1. 读取图像（处理路径转义，增加读取失败判断）
    img_road = cv2.imread(image_path)
    if img_road is None:
        print(f"错误：无法读取图片，请检查路径是否正确 -> {image_path}")
        return
    
    # 2. 预处理：灰度化 + 边缘检测
    gray_road = cv2.cvtColor(img_road, cv2.COLOR_BGR2GRAY)
    edges_road = cv2.Canny(gray_road, threshold1=50, threshold2=150, apertureSize=3)
    
    # 3. 霍夫直线变换（概率霍夫线，适合车道线检测）
    lines = cv2.HoughLinesP(
        edges_road,
        rho=1,               # 像素精度
        theta=np.pi/180,     # 角度精度（弧度）
        threshold=80,        # 检测阈值（可根据图片调整）
        minLineLength=80,    # 直线最小长度（过滤短线段）
        maxLineGap=15        # 线段间最大间隙（允许拼接的间隙）
    )
    
    # 4. 绘制直线（判断是否检测到直线，避免空指针）
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img_road, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 绿色直线
        print(f"成功检测到 {len(lines)} 条直线")
    else:
        print("未检测到直线，可尝试调整 HoughLinesP 的参数")
    
    # 5. 显示结果
    cv2.imshow("Road Lines Detection", img_road)
    cv2.imshow("Canny Edges", edges_road)  # 显示边缘检测结果，方便调参
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# -------------------------- 二、零件图片圆孔尺寸测量（修复路径+读取判断）--------------------------
def measure_circle_size(image_path, ref_real=50.0, ref_pixel=200):
    # 参数说明：ref_real=参考长度的实际值（mm），ref_pixel=参考长度的像素数
    # 1. 读取图像（处理路径转义，增加读取失败判断）
    img_part = cv2.imread(image_path)
    if img_part is None:
        print(f"错误：无法读取图片，请检查路径是否正确 -> {image_path}")
        return
    
    # 2. 预处理：灰度化 + 高斯模糊（减少噪声，提升圆检测精度）
    gray_part = cv2.cvtColor(img_part, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray_part, (5, 5), 0)
    
    # 3. 霍夫圆变换（适合检测圆形零件）
    circles = cv2.HoughCircles(
        blur,
        method=cv2.HOUGH_GRADIENT,
        dp=1.2,              # 分辨率比例（比1稍大，平衡速度和精度）
        minDist=60,          # 圆之间的最小距离（避免重复检测）
        param1=60,           # Canny边缘检测的高阈值
        param2=35,           # 圆心检测阈值（越高越严格）
        minRadius=10,        # 最小圆半径（过滤小噪声）
        maxRadius=150        # 最大圆半径（根据零件圆孔大小调整）
    )
    
    # 4. 尺寸标定（像素 -> 实际毫米）
    scale = ref_real / ref_pixel  # 比例因子：毫米/像素
    
    # 5. 绘制圆并计算尺寸（判断是否检测到圆）
    if circles is not None:
        circles = np.uint16(np.around(circles))  # 圆参数取整
        for i, circle in enumerate(circles[0, :]):
            # 绘制圆心（绿色小点）和圆轮廓（红色实线）
            cv2.circle(img_part, (circle[0], circle[1]), 3, (0, 255, 0), -1)  # 圆心
            cv2.circle(img_part, (circle[0], circle[1]), circle[2], (0, 0, 255), 2)  # 圆轮廓
            
            # 计算实际直径和半径
            radius_real = circle[2] * scale  # 实际半径（mm）
            diameter_real = radius_real * 2   # 实际直径（mm）
            print(f"圆孔 {i+1} - 半径：{radius_real:.2f} mm，直径：{diameter_real:.2f} mm")
            
            # 在图片上标注尺寸（方便直观查看）
            cv2.putText(
                img_part,
                f"D={diameter_real:.1f}mm",
                (circle[0]-50, circle[1]-20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2
            )
        print(f"共检测到 {len(circles[0])} 个圆孔")
    else:
        print("未检测到圆孔，可尝试调整 HoughCircles 的参数（如 param2、minRadius、maxRadius）")
    
    # 6. 显示结果
    cv2.imshow("Part Circle Measurement", img_part)
    cv2.imshow("Blurred Image", blur)  # 显示模糊后的图像，方便调参
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# -------------------------- 调用函数（修改为你的图片路径）--------------------------
if __name__ == "__main__":
    # 1. 直线检测（替换为你的公路图片路径）
    road_image_path = r"F:\My-study\machine vision\data\2.jpg"  # 加 r 表示原始字符串，避免转义
    detect_road_lines(road_image_path)
    
    # 2. 圆孔测量（替换为你的零件图片路径，同时调整参考尺寸参数）
    # part_image_path = r"F:\My-study\machine vision\data\part.jpg"
    # measure_circle_size(
    #     image_path=part_image_path,
    #     ref_real=50.0,  # 假设零件上某参考长度的实际值（如50mm）
    #     ref_pixel=200   # 该参考长度在图片中的像素数（需自己测量）
    # )

成功检测到 35 条直线


In [ ]:
import cv2
import numpy as np

def measure_circle_size():
    # -------------------------- 1. 图片路径（务必替换为你的实际路径）--------------------------
    image_path = r"F:\My-study\machine vision\data\1.jpg"  # 示例路径，需替换为你的零件图片路径

    # -------------------------- 2. 读取图片并校验 --------------------------
    img_part = cv2.imread(image_path)
    if img_part is None:
        print(f"❌ 图片读取失败！请检查路径：{image_path}")
        return

    # -------------------------- 3. 图像预处理 --------------------------
    gray_part = cv2.cvtColor(img_part, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray_part, (5, 5), 0)

    # -------------------------- 4. 霍夫圆变换（针对两个圆孔优化参数）--------------------------
    circles = cv2.HoughCircles(
        blur,
        method=cv2.HOUGH_GRADIENT,
        dp=1.2,              # 分辨率比例
        minDist=80,          # 两个圆心的最小距离（避免误判，根据两圆实际间距调整）
        param1=60,           # Canny高阈值
        param2=30,           # 降低阈值以确保两个圆都被检测（原35→30）
        minRadius=20,        # 最小圆半径（适配小圆）
        maxRadius=80         # 最大圆半径（适配大圆）
    )

    # -------------------------- 5. 尺寸标定（核心！必须根据实际零件校准）--------------------------
    # 步骤：在图片中找一个**已知实际长度**的参考物（如零件的某条边），测量其**像素数**
    # 示例：若零件上某条边实际长 40mm，在图片中像素数为 160，则：
    ref_real = 40.0   # 参考物实际长度（单位：mm）
    ref_pixel = 160   # 参考物的像素数
    scale = ref_real / ref_pixel  # 1像素 = scale 毫米

    # -------------------------- 6. 检测并绘制圆孔，计算尺寸 --------------------------
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i, (x, y, r) in enumerate(circles[0]):
            # 绘制圆心和圆轮廓
            cv2.circle(img_part, (x, y), 3, (0, 255, 0), -1)  # 绿色圆心
            cv2.circle(img_part, (x, y), r, (0, 0, 255), 2)    # 红色圆轮廓

            # 计算实际尺寸
            radius = r * scale
            diameter = radius * 2
            print(f"🔍 圆孔 {i+1} - 半径：{radius:.2f} mm，直径：{diameter:.2f} mm")

            # 标注尺寸到图片上
            cv2.putText(
                img_part,
                f"D={diameter:.1f}mm",
                (x - 60, y - 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (255, 0, 0),
                2
            )
        print(f"\n✅ 共检测到 {len(circles[0])} 个圆孔")
    else:
        print("⚠️  未检测到圆孔！可尝试调整 param2（再降低）、minRadius/maxRadius（扩大范围）")

    # -------------------------- 7. 显示结果 --------------------------
    cv2.imshow("圆孔尺寸测量", img_part)
    cv2.imshow("预处理（模糊）", blur)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    measure_circle_size()


✅ 最终检测到 0 个真实圆孔
